<a href="https://colab.research.google.com/github/anselmo-pitombeira/Notebooks/blob/master/Problema_de_alocacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Instalação do pyomo e do solver para o colab ##
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc

In [ ]:
## Modelo matemático ##
## Min x11 + 4x12 + 6x13 + 3x14 + 9x21 + 7x22 + 10x23 + 9x24 + 4x31 + 5x32 + 11x33 + 7x34 +8x41 + 7x42 + 8x43 + 5x44 ##
##                               ##
## S.a x11 + x12 + x13 + x14 = 1 ##
##     x21 + x22 + x23 + x24 = 1 ##
##     x31 + x32 + x33 + x34 = 1 ##
##     x41 + x42 + x43 + x44 = 1 ##
##                               ##
##     x11 + x21 + x31 + x41 = 1 ##
##     x12 + x22 + x32 + x42 = 1 ##
##     x13 + x23 + x33 + x43 = 1 ##
##     x14 + x24 + x34 + x44 = 1 ##


## Importando a biblioteca pyomo ##
import pyomo.environ as pyEnv

## Dados do problema ##
custos = [[1, 4, 6, 3], [9, 7, 10, 9], [4, 5, 11, 7], [8, 7, 8, 5]]
m = len(custos[0])
n = len(custos[0])

## Declarando o modelo através da biblioteca ##
modelo = pyEnv.ConcreteModel()

## Criando Indices ##
modelo.Indices = pyEnv.RangeSet(m)
modelo.Indices2 = pyEnv.RangeSet(n)

## Criando as variáveis ##
modelo.x = pyEnv.Var(modelo.Indices, modelo.Indices2, within = pyEnv.NonNegativeReals)
modelo.Custo_alocacao = pyEnv.Param(modelo.Indices, modelo.Indices2, initialize = lambda modelo, i, j: custos[i-1][j-1])

## Criando a Função Objetivo ##
def Objetivo(modelo):
  return sum(modelo.x[i,j] * modelo.Custo_alocacao[i,j] for i in modelo.Indices for j in modelo.Indices2)

modelo.Objetivo = pyEnv.Objective(rule = Objetivo, sense = pyEnv.minimize)

## Criando as restrições ##
def rest1(modelo, i):
  return sum(modelo.x[i,j] for j in modelo.Indices2) == 1

def rest2(modelo, j):
  return sum(modelo.x[i,j] for i in modelo.Indices) == 1

modelo.rest1 = pyEnv.Constraint(modelo.Indices, rule = rest1)
modelo.rest2 = pyEnv.Constraint(modelo.Indices2, rule = rest2)



In [ ]:
## Chamando o Solver ##
solver = pyEnv.SolverFactory('cbc')
result_objetivo = solver.solve(modelo, tee = True)

In [ ]:
## Printando o resultado ##
lista = list(modelo.x.keys())
print("Alocação ótima:\n")
for i in lista:
  if modelo.x[i]() != 0:
    print(i, '---', modelo.x[i]())
print()
print('Valor da função objetivo (custo mínimo) =', modelo.Objetivo())